In [2]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import chain, combinations
from scipy.linalg import det

In [3]:
def bernoulli_trial(p):
    return np.random.rand() < p

def poulson_dpp_sampling(K):
    n = K.shape[0]
    sample = []
    A = np.array(K, dtype=float)

    for j in range(n):
        if bernoulli_trial(A[j, j]):
            sample.append(j+1)
        else:
            A[j, j] -= 1 
        A[j+1:n, j] /= A[j, j]  
        A[j+1:n, j+1:n] -= np.outer(A[j+1:n, j], A[j, j+1:n])
    return sample, A


In [4]:
# Generate a random 3x3 matrix A
np.random.seed(42)
A = np.random.randn(3, 3)
K = A @ A.T  
eigenvalues, _ = np.linalg.eig(K)
largest_eigenvalue = max(eigenvalues)
K = K/ (2 * largest_eigenvalue)
I = np.eye(3)
L = K @ np.linalg.inv(I - K)
print("---K Matrix---")
print(K)
print("---L Matrix---")
print(L)


---K Matrix---
[[0.06409694 0.05959793 0.03500052]
 [0.05959793 0.22719832 0.21842042]
 [0.03500052 0.21842042 0.30894024]]
---L Matrix---
[[0.07870695 0.10830563 0.08886564]
 [0.10830563 0.43180019 0.45802863]
 [0.08886564 0.45802863 0.59632092]]


In [5]:
def powerset(iterable):
    "powerset([1,2,3]) --> [], [1], [2], [3], [1,2], [1,3], [2,3], [1,2,3]"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [6]:
# Calculate the powerset of indices, including the empty set
indices = [1, 2, 3]
ps = list(powerset(indices))
ps

[(), (1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]

In [7]:
det_submatrices = {p: (det(K[np.ix_([i-1 for i in p], [j-1 for j in p])]) if p else 1.0) for p in ps}
print(det_submatrices.keys())
print("---Det of K---")
for subset, determinant in det_submatrices.items():
    subset_str = ", ".join(map(str, subset))  
    print(f"Subset ({subset_str}): {determinant}")
print("----Probabilities using L Matrix---")
probs = {p: (det(L[np.ix_([i-1 for i in p], [j-1 for j in p])]) if p else 1.0)/det(I+L) for p in ps}

sorted_probabilities = sorted(probs.items(), key=lambda x: (len(x[0]), x[0]))
print("Total Probabilities: " + str(sum(probs.values())))
for subset, probability in sorted_probabilities:
    subset_formatted = f"({', '.join(map(str, subset))})" if subset else "()"
    print(f"Subset {subset_formatted}: {probability:.6f}")


dict_keys([(), (1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)])
---Det of K---
Subset (): 1.0
Subset (1): 0.06409694200724679
Subset (2): 0.2271983159436042
Subset (3): 0.30894024298493544
Subset (1, 2): 0.011010804037692794
Subset (1, 3): 0.018577088347950948
Subset (2, 3): 0.022483223285009585
Subset (1, 2, 3): 0.0009766826013799937
----Probabilities using L Matrix---
Total Probabilities: 1.0
Subset (): 0.450859
Subset (1): 0.035486
Subset (2): 0.194681
Subset (3): 0.268857
Subset (1, 2): 0.010034
Subset (1, 3): 0.017600
Subset (2, 3): 0.021507
Subset (1, 2, 3): 0.000977


In [8]:
def DPP_sampling(iterations):
    final_results = defaultdict(int)

    for _ in range(iterations):
        sample, _ = poulson_dpp_sampling(K) 
        final_results[tuple(sample)] += 1

    probabilities = {subset: count / iterations for subset, count in final_results.items()}
    sorted_probabilities = dict(sorted(probabilities.items(), key=lambda item: (len(item[0]), item)))
    print("Total Probabilities: " + str(sum(probabilities.values())))
    for subset, probability in sorted_probabilities.items():
        subset_formatted = f"({', '.join(map(str, subset))})" if subset else "()"
        print(f"Subset {subset_formatted}: {probability:.6f}")

    return [probability for subset, probability in sorted_probabilities.items()]
2
iterations = 10000000
probabilities = DPP_sampling(iterations)

Total Probabilities: 0.9999999999999999
Subset (): 0.450906
Subset (1): 0.035495
Subset (2): 0.194575
Subset (3): 0.268916
Subset (1, 2): 0.010004
Subset (1, 3): 0.017610
Subset (2, 3): 0.021530
Subset (1, 2, 3): 0.000963
